In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import io
import base64

In [2]:
# Load model and labels
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'frozen_inference_graph.pb'
model = cv2.dnn_DetectionModel(frozen_model, config_file)
model.setInputSize(320, 320)
model.setInputScale(1.0 / 127.5) # 255/2=127.5
model.setInputMean((127.5, 127.5, 127.5)) # mobilenet => [-1,1]
model.setInputSwapRB(True) # auto convert BGR to RGB

classLabels = []
file_name = 'Labels.txt'
with open(file_name, 'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')
    # print(classLabels)

In [3]:
# Function to detect objects in an image
def detect_objects_in_image(change):
    clear_output()
    display(image_button, video_button, webcam_button)
    
    if change['new']:
        uploaded_file = change['owner']
        image_name = list(uploaded_file.keys())[0]
        image_data = uploaded_file[image_name]['content']
        image = np.frombuffer(image_data, np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        
        # Perform object detection on the image
        ClassIndex, confidence, bbox = model.detect(image, confThreshold=0.55)
        
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
            if ClassInd <= 80:
                cv2.rectangle(image, (boxes[0], boxes[1]), (boxes[0]+boxes[2], boxes[1]+boxes[3]), (255, 0, 0), 2)
                cv2.putText(image, classLabels[ClassInd-1], (boxes[0], boxes[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Convert image to base64 encoding
        _, buffer = cv2.imencode('.jpg', image)
        img_str = base64.b64encode(buffer).decode('utf-8')
        
        # Display the name of the uploaded image
        display(widgets.Label(value=f"Uploaded Image: {image_name}"))
        
        # Display annotated image
        display(Image(data=buffer, format='jpg'))

# Function to detect objects in a video
def detect_objects_in_video(change):
    # clear_output()
    display(image_button, video_button, webcam_button)
    
    if change['new']:
        uploaded_file = change['owner']
        video_path = io.BytesIO(uploaded_file['content'])
        
        # Perform object detection on the video
        cap = cv2.VideoCapture(video_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)
            
            for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
                if ClassInd <= 80:
                    cv2.rectangle(frame, boxes, (255, 0, 0), 2)
                    cv2.putText(frame, classLabels[ClassInd-1], (boxes[0]+10, boxes[1]+40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
            
            cv2.imshow('Object Detection in Video', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

# Function to detect objects in webcam
def detect_objects_in_webcam(change):
    clear_output()
    display(image_button, video_button, webcam_button)
    
    if change['new']:
        cap = cv2.VideoCapture(0)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)
            
            for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
                if ClassInd <= 80:
                    cv2.rectangle(frame, boxes, (255, 0, 0), 2)
                    cv2.putText(frame, classLabels[ClassInd-1], (boxes[0]+10, boxes[1]+40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)
            
            cv2.imshow('Object Detection in Webcam', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

# Create buttons for image, video, and webcam
image_button = widgets.FileUpload(accept='.jpeg, .jpg, .png', description='Image',multiple=False)
video_button = widgets.FileUpload(accept='.mp4', description='Video',multiple=False)
webcam_button = widgets.Button(description='Webcam')

# Attach event handlers to the buttons
image_button.observe(detect_objects_in_image, names='value')
video_button.observe(detect_objects_in_video, names='value')
webcam_button.on_click(detect_objects_in_webcam)

# Display the buttons
display(image_button, video_button, webcam_button)

FileUpload(value=(), accept='.jpeg, .jpg, .png', description='Image')

FileUpload(value=(), accept='.mp4', description='Video')

Button(description='Webcam', style=ButtonStyle())

In [21]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64
import time

# Function to handle the selection of the toggle buttons
def handle_toggle_change(change):
    if change['new'] == 'Image':
        clear_output()
        display(image_upload_button, detect_button, output)
    elif change['new'] == 'Video':
        clear_output()
        print("Video selected")
        # Implement video related functionality here
    elif change['new'] == 'Webcam':
        clear_output()
        print("Webcam selected")
        # Implement webcam related functionality here

# Function to handle image upload
def handle_image_upload(change):
    if len(change['new']) > 0:
        uploaded_file = change['new']
        
        # Display loader
        with output:
            clear_output()
            display(loader)
        
        # Simulate progress with sleep
        for i in range(101):
            loader.value = i
            time.sleep(0.02)  # Sleep for 20 milliseconds for smooth progression
        
        # Extract image content
        image_content = uploaded_file[list(uploaded_file.keys())[0]]['content']
        
        # Display the uploaded image
        img_widget = widgets.Image(value=image_content)
        
        # Display the image
        with output:
            clear_output()
            display(img_widget)

# Function to handle detect button click
def handle_detect_button_click(b):
    # Display the uploaded image
    with output:
        clear_output()
        uploaded_files = image_upload_button.value
        if len(uploaded_files) > 0:
            uploaded_file = uploaded_files[0]  # Select the first file in the tuple
            image_content = uploaded_file['content']
            img_widget = widgets.Image(value=image_content)
            display(img_widget)


# Create toggle buttons
toggle_buttons = widgets.ToggleButtons(
    options=['Image'],
    description='Select:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Upload Image', 'Select Video', 'Use Webcam']
)

# Create image upload button
image_upload_button = widgets.FileUpload(accept='.jpeg, .jpg, .png', description='Upload Image')

# Create detect button
detect_button = widgets.Button(description='Detect')
detect_button.on_click(handle_detect_button_click)

# Create loader
loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Loading:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

# Create output widget
output = widgets.Output()

# Attach event handlers to the toggle buttons and image upload button
toggle_buttons.observe(handle_toggle_change, names='value')
image_upload_button.observe(handle_image_upload, names='value')

# Display the toggle buttons, image upload button, detect button, and output widget
display(widgets.VBox([toggle_buttons, image_upload_button, detect_button, output]))


In [24]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import base64
import time

# Function to handle image upload
def handle_image_upload(change):
    if len(change['new']) > 0:
        uploaded_file = change['new']
        
        # Display loader
        with output:
            clear_output()
            display(loader)
        
        # Simulate progress with sleep
        for i in range(101):
            loader.value = i
            time.sleep(0.01)  # Sleep for 10 milliseconds for smooth progression
        
        # Extract image content
        image_content = uploaded_file[list(uploaded_file.keys())[0]]['content']
        
        # Display the uploaded image
        img_widget = widgets.Image(value=image_content)
        
        # Display the image
        with output:
            clear_output()
            display(img_widget)

# Function to handle detect button click
def handle_detect_button_click(b):
    # Display the uploaded image
    with output:
        clear_output()
        uploaded_files = image_upload_button.value
        if len(uploaded_files) > 0:
            uploaded_file = uploaded_files[0]  # Select the first file in the tuple
            image_content = uploaded_file['content']
            img_widget = widgets.Image(value=image_content)
            display(img_widget)

# Create image upload button
image_upload_button = widgets.FileUpload(accept='.jpeg, .jpg, .png', description='Upload Image')

# Create detect button
detect_button = button = widgets.Button(
    description='Detect',
    disabled=False,
    button_style='Success',
    tooltip='Detect',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
detect_button.on_click(handle_detect_button_click)

# Create loader
loader = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Loading:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

# Create output widget
output = widgets.Output()

# Create container widget for image-related components
image_tab_content = widgets.VBox([image_upload_button, detect_button, output])

# Create Tab widget
tab_contents = ['Image', 'Video', 'Webcam']
tabs = widgets.Tab()
tabs.children = [image_tab_content, widgets.Text('Video Placeholder'), widgets.Text('Webcam Placeholder')]
for i, tab_name in enumerate(tab_contents):
    tabs.set_title(i, tab_name)

# Attach event handlers to the image upload button
image_upload_button.observe(handle_image_upload, names='value')

# Display the Tab widget
display(tabs)
